<h2>Sequence-to-Sequence</h2>
<b>Sequence-to-Sequence(Seq2Seq) 는 입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 모델, 기계 번역 등</b><br>
<b>RNN 기술을 조합해 만들며, Encoder와 Decoder로 구성</b>

In [1]:
import os
import shutil
import urllib3
import zipfile
import pandas as pd

In [2]:
http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)

with http.request('GET', url, preload_content = False) as r, open(zipfilename, 'wb') as out_file:
    shutil.copyfileobj(r, out_file)
    
with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [2]:
lines = pd.read_csv('fra.txt', names = ['src', 'tar', 'lic'], sep = '\t')
del lines['lic']
len(lines)

189986

In [3]:
lines = lines.loc[:, 'src':'tar']
lines = lines[:60000]
lines.tar = lines.tar.apply(lambda x: '\t ' + x + '\n' )
lines[:10]  

,src,tar
0,Go.,\t Va !\n
1,Go.,\t Marche.\n
2,Go.,\t Bouge !\n
3,Hi.,\t Salut !\n
4,Hi.,\t Salut.\n
5,Run!,\t Cours !\n
6,Run!,\t Courez !\n
7,Run!,\t Prenez vos jambes à vos cous !\n
8,Run!,\t File !\n
9,Run!,\t Filez !\n


<b>글자 단위로 예측하기 위해 글자 집합 구축</b><br>
<b>구축 후, 정렬해 인덱스 부여후 글자에 해당하는 사전 만듦</b><br>
<b>글자를 모델에 투입하도록 변환하거나 예측시 반환되는 인덱스를 글자로 변환할 때 사용</b><br>

In [4]:
src_vocab = set()
for line in lines.src:
    for char in line:
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)    

In [5]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1

src_to_idx = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_idx = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

print(src_to_idx)
print(tar_to_idx)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 3

<b>인코더에 입력될 데이터 구성, 글자 하나씩을 사전을 이용해 인덱스로 변환해 리스트에 넣음</b>

In [6]:
encoder_input = []
for line in lines.src:
    encoder_input.append([src_to_idx[w] for w in line])
    
print(encoder_input[:5])

[[30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10], [31, 58, 10]]


<b>디코더에 입력될 데이터 구성, 목표 데이터에 해당하는 사전을 사용해야 함</b>

In [7]:
decoder_input = []
for line in lines.tar:
    decoder_input.append([tar_to_idx[w] for w in line])
    
print(decoder_input[:5])

[[1, 3, 48, 53, 3, 4, 2], [1, 3, 39, 53, 70, 55, 60, 57, 14, 2], [1, 3, 28, 67, 73, 59, 57, 3, 4, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 2], [1, 3, 45, 53, 64, 73, 72, 14, 2]]


<b>디코더 출력과 비교할 목표 데이터 구성, 디코더와 동일하나 시작 토큰 제외</b>

In [8]:
decoder_target = []
for line in lines.tar:
    decoder_target.append([tar_to_idx[w] for w in line if w != '\t'])
    
print(decoder_target[:5])

[[3, 48, 53, 3, 4, 2], [3, 39, 53, 70, 55, 60, 57, 14, 2], [3, 28, 67, 73, 59, 57, 3, 4, 2], [3, 45, 53, 64, 73, 72, 3, 4, 2], [3, 45, 53, 64, 73, 72, 14, 2]]


In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])

encoder_input = pad_sequences(encoder_input, maxlen = max_src_len, padding = 'post')
decoder_input = pad_sequences(decoder_input, maxlen = max_tar_len, padding = 'post')
decoder_target = pad_sequences(decoder_target, maxlen = max_tar_len, padding = 'post')

In [10]:
from tensorflow.keras.utils import to_categorical

encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

<h2>인코더 모델 구성</h2>
<b>Encoder 구성은 일반 LSTM 모델과 동일</b><br>
<b>LSTM 안의 return_state는 은닉 상태를 반환해줘 seq2seq 모델을 구성할 때 필요</b>

In [11]:
from keras.layers import Input, LSTM

In [12]:
encoder_inputs = Input(shape = (None, src_vocab_size))
encoder_lstm = LSTM(256, return_state = True)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

<h2>디코더 모델 구성</h2>
<b>모델 구성은 Encoder와 거의 유사</b><br>
<b>LSTM 안의 return_sequences는 출력을 시퀀스로 반환할 때 사용</b><br>
<b>decoder_lstm 사용시 initial_state를 인코더의 은닉 상태로 설정</b><br>
<b>마지막으로 Dense layer와 softmax를 통과해 예측 글자에 해당하는 인덱스를 반환</b><br>

In [13]:
from keras.layers import Dense

decoder_inputs = Input(shape = (None, tar_vocab_size))
decoder_lstm = LSTM(256, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation = 'softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

<h2>Seq2Seq 모델</h2>
<b>Encoder와 Decoder를 결합해 Seq2Seq model 구성</b><br>
<b>구성한 모델과 준비한 데이터를 사용해 기계 번역 학습</b>

In [14]:
from keras.models import Model

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy')

In [15]:
model.fit(x = [encoder_input, decoder_input], y = decoder_target,
         batch_size = 128,
         epochs = 25,
         validation_split = 0.2)

Epoch 1/25
375/375 [==============================] - 320s 770ms/step - loss: 1.2242 - val_loss: 0.7898
Epoch 2/25
375/375 [==============================] - 271s 722ms/step - loss: 0.5864 - val_loss: 0.6556
Epoch 3/25
375/375 [==============================] - 198s 529ms/step - loss: 0.4825 - val_loss: 0.5594
Epoch 4/25
375/375 [==============================] - 189s 503ms/step - loss: 0.4195 - val_loss: 0.5045
Epoch 5/25
375/375 [==============================] - 189s 505ms/step - loss: 0.3784 - val_loss: 0.4712
Epoch 6/25
375/375 [==============================] - 190s 507ms/step - loss: 0.3501 - val_loss: 0.4452
Epoch 7/25
375/375 [==============================] - 188s 500ms/step - loss: 0.3260 - val_loss: 0.4239
Epoch 8/25
375/375 [==============================] - 195s 521ms/step - loss: 0.3092 - val_loss: 0.4122
Epoch 9/25
375/375 [==============================] - 190s 507ms/step - loss: 0.2934 - val_loss: 0.4003
Epoch 10/25
375/375 [==============================] - 189s 504m

<h2>예측</h2>
<b>일반 모델과 달리 seq2seq는 모델 예측 프로세스가 다름</b><br>
<b>예측할 때는 인덱스를 하나씩 예측, 예측한 인덱스를 저장하고 다시 입력으로 사용해 종료 토큰이 나올 때까지 반복</b><br>
<b>마지막으로 예측한 인덱스들을 사전을 통해 글자로 변환해 최종 예측을 얻음</b><br>

In [31]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)

In [32]:
decoder_state_input_h = Input(shape = (256))
decoder_state_input_c = Input(shape = (256))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)

decoder_states = [state_h, state_c]

decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs = [decoder_inputs] + decoder_states_inputs,
                     outputs = [decoder_outputs] + decoder_states)

In [33]:
idx_to_src = dict((i, char) for char, i in src_to_idx.items())
idx_to_tar = dict((i, char) for char, i in tar_to_idx.items())

In [37]:
def predict_decode(input_seq):
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_idx['\t']] = 1
    
    stop = False
    decoded_sentence = ""
    
    while not stop:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx_to_tar[sampled_token_index]
        
        decoded_sentence += sampled_char
        
        if sampled_char == '\n' or len(decoded_sentence) > max_tar_len:
            stop = True
            
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.
        
        states_values = [h, c]
        
    return decoded_sentence    

In [35]:
import numpy as np

In [38]:
for seq_index in [100, 200, 300, 400]:
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = predict_decode(input_seq)
    
    print("입력: ", lines.src[seq_index])
    print("정답: ", lines.tar[seq_index][1:len(lines.tar[seq_index]) - 1])
    print("번역: ", decoded_sentence[:len(decoded_sentence) - 1], '\n')

입력:  Hug me.
정답:   Serrez-moi dans vos bras !
번역:                                                                              

입력:  Come in.
정답:   Entre.
번역:                                                                              

입력:  Hold it!
정답:   Restez où vous êtes !
번역:                                                                              

입력:  Tom won.
정답:   Tom a gagné.
번역:                                                                              



<b>seq2seq 모델은 하나의 고정 길이 벡터에 모든 정보를 압축해 정보 손실 발생</b><br>
<b>RNN의 기울기 소실 문제가 똑같이 발생</b><br>
<b>Attention Mechanism으로 보완, 디코더가 예측하는 시점마다 인코더의 전체 입력 문장을 다시 참조</b><br>

<b>Encoder</b>

In [40]:
encoder_inputs = Input(shape = (None, src_vocab_size))
encoder_lstm = LSTM(256, return_state = True)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

<b>Decoder</b>

In [41]:
import tensorflow as tf
from keras.layers import Attention

In [42]:
decoder_inputs = Input(shape = (None, tar_vocab_size))
decoder_lstm = LSTM(256, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs[:, :-1, :]], axis = 1)

attention = Attention()
context_vector = attention([S_, encoder_outputs])
concat = tf.concat([decoder_outputs, context_vector], axis = -1)
decoder_softmax_layer = Dense(tar_vocab_size, activation = 'softmax')
decoder_outputs = decoder_softmax_layer(concat)

<b>Seq2Seq와 동일한 model 구성 방법</b><br>
<b>Attention mechanism 활용 후 학습 시간이 절반 가량으로 줄어듦</b>

In [43]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy')

In [44]:
model.fit(x = [encoder_input, decoder_input], 
          y = decoder_target,
          batch_size = 128,
          epochs = 25,
          validation_split = 0.2)

Epoch 1/25
375/375 [==============================] - 335s 869ms/step - loss: 1.1627 - val_loss: 0.7853
Epoch 2/25
375/375 [==============================] - 350s 934ms/step - loss: 0.5819 - val_loss: 0.6455
Epoch 3/25
375/375 [==============================] - 506s 1s/step - loss: 0.4781 - val_loss: 0.5712
Epoch 4/25
375/375 [==============================] - 445s 1s/step - loss: 0.4128 - val_loss: 0.5202
Epoch 5/25
375/375 [==============================] - 412s 1s/step - loss: 0.3723 - val_loss: 0.4938
Epoch 6/25
375/375 [==============================] - 237s 632ms/step - loss: 0.3444 - val_loss: 0.4696
Epoch 7/25
375/375 [==============================] - 238s 636ms/step - loss: 0.3214 - val_loss: 0.4563
Epoch 8/25
375/375 [==============================] - 235s 627ms/step - loss: 0.3055 - val_loss: 0.4381
Epoch 9/25
375/375 [==============================] - 233s 621ms/step - loss: 0.2896 - val_loss: 0.4320
Epoch 10/25
375/375 [==============================] - 234s 625ms/step - 

<h2>예측</h2>
<b>seq2seq과 동일하나, 추가된 모델 구조를 반영해줘야함(Attention layer)</b><br>
<b>Encoder와 Decoder를 분리하였기에 디코더에서 은닉 상태(estate_h)와 최종 은닉 상태(encoder_outputs)를 따로 입력 받아야함</b><br>estate_h

In [45]:
encoder_model = Model(inputs = encoder_inputs, 
                      outputs =[encoder_outputs, encoder_states])

In [46]:
decoder_state_input_h = Input(shape = (256))
decoder_state_input_c = Input(shape = (256))
estate_h = Input(shape = (256))
encoder_outputs = Input(shape = (256))


decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)

decoder_states = [state_h, state_c]

S_ = tf.concat([estate_h[:, tf.newaxis, :], decoder_outputs[:, :-1, :]], axis = 1)
context_vector = attention([S_, encoder_outputs])
decoder_concat = tf.concat([decoder_outputs, context_vector], axis = -1)
decoder_outputs = decoder_softmax_layer(decoder_concat)
decoder_model = Model(inputs = [decoder_inputs, estate_h, encoder_outputs] + decoder_states_inputs,
                     outputs = [decoder_outputs] + decoder_states)

In [47]:
idx_to_src = dict((i, char) for char, i in src_to_idx.items())
idx_to_tar = dict((i, char) for char, i in tar_to_idx.items())

In [48]:
def predict_decode(input_seq):
    outputs_input, states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_idx['\t']] = 1
    
    stop = False
    decoded_sentence = ""
    
    while not stop:
        output_tokens, h, c = decoder_model.predict([target_seq, states_value[0], outputs_input] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx_to_tar[sampled_token_index]
        
        decoded_sentence += sampled_char
        
        if sampled_char == '\n' or len(decoded_sentence) > max_tar_len:
            stop = True
            
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.
        
        states_values = [h, c]
        
    return decoded_sentence    

In [49]:
for seq_index in [100, 200, 300, 400]:
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = predict_decode(input_seq)
    
    print("입력: ", lines.src[seq_index])
    print("정답: ", lines.tar[seq_index][1:len(lines.tar[seq_index]) - 1])
    print("번역: ", decoded_sentence[:len(decoded_sentence) - 1], '\n')

입력:  Hug me.
정답:   Serrez-moi dans vos bras !
번역:                                                                              

입력:  Come in.
정답:   Entre.
번역:                                                                              

입력:  Hold it!
정답:   Restez où vous êtes !
번역:                                                                              

입력:  Tom won.
정답:   Tom a gagné.
번역:                                                                              

